## Methanol-Synthese

T=493;15K

p=50 bar

### Bilanzen
Stoffbilanzen

$\dot n_i = \dot n_{i, 0} + \sum_{j}{\nu_ij \xi_j}$ 

Energiebilanz

$0 = \dot Q + \sum\limits_{j}{\xi_j \Delta H_j}$

### Gleichgewichtskonstanten

$\begin{array}{ll}
exp \left(- \frac{\Delta G_i}{R T} \right) &= K_p K_{\phi^{eq}} = K_x \prod\limits_{i} \left( \frac{p}{p^0}\right)^{\nu_i}  K_{\phi^{eq}} \\
&=\prod\limits_{i} (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum\limits_{i} \nu_i}(n)^{-\sum\limits_{i} \nu_i}   K_{\phi^{eq}}\end{array}$

$p^° = 1 bar$

Idealer Gas, $K_{\phi^{eq}}=1$ 

**Methode A)** Geringe Veränderung der Reaktionsenthalpie mit der Temperatur

Van't Hoff, $\frac{d ln K}{dT} = -\frac{\Delta H}{R T^2} \sim \Rightarrow ln \left(\frac{K}{K'} \right) = -\frac{\Delta H^0}{R}\left(\frac{1}{T} - \frac{1}{T'} \right)$

$\begin{array}{ll}
K_{(493,15K)} &= K_{(298,15K)} \times exp \left[-\frac{\Delta H^0}{R}\left(\frac{1}{493,15 K} - \frac{1}{298,15 K} \right)\right] \\
&= \prod_i (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum_i \nu_i}(n)^{-\sum_i \nu_i}\end{array}$ 

**Methode B)** Wechselwirkung der Reaktionsenthalpie mit der Temperatur [SVNA]

$\Delta H^° = \Delta H_0^° + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT}$

$\Delta S^° = \Delta S_0^° + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}$

$\Delta G^° = \Delta H^° - T \Delta S^° = \Delta H_0^° + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT} - T \Delta S_0^° - R T \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}$

$\Delta S_0^° = \frac{\Delta H_0^° - \Delta G_0^°}{T_0}$

$\Delta G^° = \Delta H_0^° - \frac{T}{T_0}(\Delta H_0^° -\Delta G_0^°) + R \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT} - R T \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}$

$\begin{array}{ll}
K_{(T)} &= exp \left(-\frac{\Delta H_0^°}{R T} + \frac{(\Delta H_0^° -\Delta G_0^°)}{R T_0} - \frac{1}{T}\int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}dT} + \int\limits_{T_0}^{T}{\frac{\Delta Cp^0}{R}\frac{dT}{T}}\right) \\
&= \prod_i (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum_i \nu_i}(n)^{-\sum_i \nu_i}\end{array}$ 

Somit läßt sich K(T) bestimmen, insofern man über einen Ausdruck für $Cp_i(T)$ verfügt. Bei geringer Veränderung der Wärmekapazität Cp im Temperatur-Bereich kann man auch einen bestimmten Mittelwert als ~konstant einsetzen.

**Methode C)** Gibbssche Energie-Funktion - Gef(T) - aus Thermochemischen Tabellen [BP]

$-Gef(T) = -[G(T)-H(298,15)]/T$

$-R ln(K) = \sum\nu_i Gef_i - \sum \nu_i H_i(298,15K)/T$

In thermochemischen Tabellen [BP] sind die Werte -Gef(T) verfügbar. 

### Literaturhinweise
* [SVNA] Smith J.M., Van Ness H.C., Abbott M.M.; Introduction to chemical engineering thermodynamics; 6th ed.; McGraw-Hill; New York; 2001; S. 458-462.
* [BP] Barin Isan, Platzki Gregor; Thermochemical data of pure substances; 3. ed.; VCH; New York; 1995.

In [17]:
from scipy import optimize
import numpy as np

p = 50. # bar
temp = 273.15 + 220. # K

namen = ['CO', 'H2', 'CO2', 'H2O', 'CH3OH']

n0co = 50.
n0h2 = 100.
n0co2 = 10.
n0h2o = 0.
n0ch3oh = 0.

ne = np.array([n0co, n0h2, n0co2, n0h2o, n0ch3oh])

nuij = np.array([[-1, -2, 0, 0, +1] ,
                 [0, -3, -1, +1, +1], 
                 [-1, +1, +1, -1, 0]]).T

h_298 = np.array(
    [-110.541, 0., -393.505, -241.826,-201.167]) * 1000 # J/mol

g_298 = np.array(
    [-169.474, -38.962, -457.240, -298.164, -272.667]) * 1000 # J/mol

cp_298 = np.array(
    [29.140, 28.836, 37.132, 33.590, 43.812] # J/(mol K)
)

cp_493 = np.array(
    [29.763, 29.254, 44.399, 35.163, 58.951] #  J/(mol K)
)

# Mean delta Cp assumed constant.
mean_cp = np.mean(
    np.array([cp_298, cp_493]), axis=0) 

# Calc. H(T) and G(T) with constant mean Cp approach (0.32% error)

h_493 = h_298 + mean_cp * (493.15 - 298.15)

g_493 = h_298 - 493.15/298.15 * (
    h_298 - g_298) + mean_cp * ((
    493.15 - 298.15) - 493.15 * np.log(493.15/298.15))

delta_hr_298 = nuij.T.dot(h_298)

delta_gr_298 = nuij.T.dot(g_298)

delta_hr_493 = nuij.T.dot(h_493)

delta_gr_493 = nuij.T.dot(g_493) 

k_298 = np.exp(-delta_gr_298/(8.314*298.15))
k_493 = np.exp(-delta_gr_493/(8.314*493.15))

for i, f in enumerate(delta_hr_298):
    print('Delta H_' + str(i+1) + '(298.15K)=' + str(f/1000.) + 'kJ/mol')

print('\n')
for i, f in enumerate(k_493):
    print('K' + str(i+1) + '(493K)=' + str(f))
print('\n')
    
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nch3oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nco + n0co - xi1
    f2 = -nh2  + n0h2 -2*xi1
    f3 = -nch3oh + n0ch3oh +xi1
    f4 = -k_493[0] * (nco * nh2**2) + \
            nch3oh * (p/1.)**-2 * (nco + nh2 + nch3oh)**-(-2)
    
    return [f1, f2, f3, f4]

n0 = np.array([n0co, n0h2, n0ch3oh])
x0 = np.append(n0, [0.])
sol = optimize.root(fun, x0)
f_final = - sol.x[:3].reshape([3,1]) + ne[[0,1,4]].reshape([3,1]) + nuij[:,0][[0,1,4]].reshape([3,1])*sol.x[-1]

print(sol)
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für nur einzige Reaktion (ohne CO2):\n')
for i, f in enumerate(sol.x[:2]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('n_' + namen[-1] + '= ' + str(sol.x[2]) + ' mol')

# Lösung des einfacheren Falls in schwierigerem Fall einwenden.
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nco2 = x_vec[2]
    nh2o = x_vec[3]
    nch3oh = x_vec[4]
    xi1 = x_vec[5]
    xi2 = x_vec[6]
    xi3 = x_vec[7]
    
    f1 = -nco + n0co - xi1 +0 -xi3
    f2 = -nh2  + n0h2 -2*xi1 -3*xi2 +xi3
    f3 = -nco2 + n0co2 +0 -xi2 +xi3
    f4 = -nh2o + n0h2o +0 +xi2 -xi3
    f5 = -nch3oh + n0ch3oh +xi1 +xi2 -0
    f6 = -k_493[0] * (nco * nh2**2) + \
        nch3oh * (p/1.)**-2 * (nco + nh2 + nco2 + nh2o + nch3oh)**-(-2)
    f7 = -k_493[1] * (nco2 * nh2**3) + \
        nch3oh * nh2o * (p/1.)**-2 * (nco + nh2 + nco2 + nh2o + nch3oh)**-(-2)
    f8 = -k_493[2] * (nco * nh2o) + \
        nco2 * nh2 * (p/1.)**0 * (nco + nh2 + nco2 + nh2o + nch3oh)**-0
    
    return [f1, f2, f3, f4, f5, f6, f7, f8]

n0 = np.array([sol.x[0], sol.x[1], n0co2, 0, sol.x[2]])
x0 = np.append(n0, [sol.x[3], 0, 0])
#n0 = np.array([1.79111141, 3.58222282, 10., 0, 48.20888859])
#x0 = np.append(n0, [48.20888859, 0, 0])
sol = optimize.root(fun, x0)

f_final = - sol.x[:5].reshape([5,1]) + ne.reshape([5,1]) + nuij.dot(sol.x[5:].reshape([3,1]))

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für alle drei Reaktionen, mit CO2:\n')
for i, f in enumerate(sol.x[:5]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')

print('\n')

for i, f in enumerate(sol.x[5:]):
    print('xi_' + str(i) + '= ' + str(f) + ' mol')

print('\n')
print('0 = Q + Sum(Delta H)_ein - Sum(Delta H)_aus')
print('-Q = -Sum(xi_j * Delta H_j) = ' + str(
    sol.x[:5].dot(h_493) - ne.dot(h_493)) + 'kJ/h')

print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

Delta H_1(298.15K)=-90.626kJ/mol
Delta H_2(298.15K)=-49.488kJ/mol
Delta H_3(298.15K)=-41.138kJ/mol


K1(493K)=0.00881414730139
K2(493K)=5.71444621084e-05
K3(493K)=154.24324556


    fjac: array([[ -2.70197849e-02,  -2.51709750e-14,  -8.58158019e-19,
         -9.99634899e-01],
       [ -3.56958949e-01,  -9.34070241e-01,  -7.06699543e-17,
          9.64847668e-03],
       [ -1.94977237e-01,   7.45658253e-02,  -9.77955030e-01,
          5.27016714e-03],
       [  9.13145179e-01,  -3.49217299e-01,  -2.08815612e-01,
         -2.46819977e-02]])
     fun: array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.42108547e-14])
 message: 'The solution converged.'
    nfev: 16
     qtf: array([  1.12812310e-08,  -1.08886449e-10,  -5.94756877e-11,
         2.78545016e-10])
       r: array([ 37.00991713, -14.14349855,  -8.45715064,   0.06650004,
         1.0705833 ,   0.08162842,   2.22471837,   1.02254191,
        -0.93231759,  -0.42255139])
  status: 1
 success: True
       x: arr

In [20]:
print('Solution, to 30 decimals')
print('')
for part in sol.x:
    print '{:.30g}'.format(part).replace('.',',')

Solution, to 30 decimals

14,8674441631637890282036096323
29,3557068387328392589097347809
9,87360617080175373416750517208
0,126393829198241464117913324117
35,2589496660344607903425639961
-92282,954122169699985533952713
293186,320238840358797460794449
27259,3708983457290742080658674


# SVN 13.6 (S. 485)
$C_2H_4(g) + H_2 O(g) \rightleftharpoons C_2H_5OH(g)$

In [4]:
p = 35. # bar
temp = 523.15 # K

namen = ['C2H4', 'H2O', 'C2H5OH']

n0c2h4 = 2.
n0h2o = 5.
n0c2h5oh = 0.

ne = np.array([n0c2h4, n0h2o, n0c2h5oh])

nuij = np.array([[-1, -1, +1]]).T

delta_h_1_298 = (-(+52.467) -(-241.826) + -234.906)*1000

delta_g_1_298 = (-(+68.420) -(-228.620) + -168.200)*1000

k1_298 = np.exp(-delta_g_1_298/(8.314*298.15))

# Angenommen, dass Delta_H fast im Interval zwischen 298.15K und 443.15K 
# unabhängig von der Temperatur sei...
k1_523 = k1_298 * np.exp(-delta_h_1_298/8.314*( 1/523.15 - 1/298.15 ))

print('\n')
for i, f in enumerate([k1_523]):
    print('K' + str(i+1) + '(523.15K)=' + str(f))
print('\n')


def fun(x_vec):    
    nc2h4 = x_vec[0]
    nh2o = x_vec[1]
    nc2h5oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nc2h4 + n0c2h4 -xi1
    f2 = -nh2o  + n0h2o -xi1
    f3 = -nc2h5oh + n0c2h5oh +xi1
    f4 = -k1_523 * (nc2h4 * nh2o) + \
            nc2h5oh * (p/1.)**-1. * (nc2h4 + nh2o + nc2h5oh)**(-(-1))
    
    return [f1, f2, f3, f4]

n0 = np.array([(0.233), n0h2o, n0c2h5oh])
x0 = np.append(n0, [0.233])
sol = optimize.root(fun, x0)
f_final = - sol.x[:3].reshape([3,1]) + n0.reshape([3,1]) + nuij.reshape([3,1])*sol.x[-1]

print(sol)
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

print('\n\n')
print('T=' + str(temp) +'K, p=' + str(p) +'bar')
for i, f in enumerate(sol.x[:-1]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('x1= ' + str(sol.x[-1]))



K1(523.15K)=0.00932292186355


    fjac: array([[ -9.99223948e-01,   2.10438579e-10,  -1.04114753e-10,
         -3.93891191e-02],
       [  5.01145052e-04,  -9.99919060e-01,  -8.25469011e-10,
         -1.27130629e-02],
       [ -6.22922226e-03,  -2.01224058e-03,  -9.87413727e-01,
          1.58023035e-01],
       [  3.88902089e-02,   1.25628023e-02,  -1.58158566e-01,
         -9.86567583e-01]])
     fun: array([  5.55111512e-17,  -1.66533454e-16,   0.00000000e+00,
         0.00000000e+00])
 message: 'The solution converged.'
    nfev: 11
     qtf: array([  2.24060002e-13,   7.19101664e-14,  -8.98671873e-13,
         5.61057783e-12])
       r: array([  1.00077665e+00,   5.01569423e-04,  -6.31454054e-03,
         9.98252337e-01,   1.00008095e+00,  -2.03805403e-03,
         9.99104324e-01,   1.01274671e+00,  -9.75274364e-01,
        -2.33946887e-01])
  status: 1
 success: True
       x: array([ 1.62885912,  4.62885912,  0.37114088,  0.37114088])



Zustand der Optimisierungs-Funktionen


# Min(G)

$\Delta G_{f i}^{°} + R T ln(y_i \hat{\phi_i} P/P^°) + \sum\limits_{k}{ \lambda_k a_{ik}}=0 \hspace{10mm} (i=1,2,...,N)$

$\sum\limits_{i}{n_i a_{ik}}=A_k \hspace{10mm} (k = 1,2,...,\omega)$

In [19]:
from scipy import optimize
import numpy as np

p = 50. # bar
temp = 273.15 + 220. # K

namen = ['CO', 'H2', 'CO2', 'H2O', 'CH3OH']

atom_namen = ['C', 'O', 'H']

n0co = 50.
n0h2 = 100.
n0co2 = 10.
n0h2o = 0.
n0ch3oh = 0.

ne = np.array([n0co, n0h2, n0co2, n0h2o, n0ch3oh])

aik = np.array([[1, 0, 1, 0, 1] ,
                 [1, 0, 2, 1, 1], 
                 [0, 2, 0, 2, 4]]).T

ak = aik.T.dot(ne)

h_298 = np.array(
    [-110.541, 0., -393.505, -241.826,-201.167]) * 1000 # J/mol

g_298 = np.array(
    [-169.474, -38.962, -457.240, -298.164, -272.667]) * 1000 # J/mol

cp_298 = np.array(
    [29.140, 28.836, 37.132, 33.590, 43.812] # J/(mol K)
)

cp_493 = np.array(
    [29.763, 29.254, 44.399, 35.163, 58.951] #  J/(mol K)
)

# Mean delta Cp assumed constant.
mean_cp = np.mean(
    np.array([cp_298, cp_493]), axis=0) 

# Calc. H(T) and G(T) with constant mean Cp approach (0.32% error)

h_493 = h_298 + mean_cp * (493.15 - 298.15)

g_493 = h_298 - 493.15/298.15 * (
    h_298 - g_298) + mean_cp * ((
    493.15 - 298.15) - 493.15 * np.log(493.15/298.15))

delta_hr_298 = nuij.T.dot(h_298)

delta_gr_298 = nuij.T.dot(g_298)

delta_hr_493 = nuij.T.dot(h_493)

delta_gr_493 = nuij.T.dot(g_493) 

k_298 = np.exp(-delta_gr_298/(8.314*298.15))
k_493 = np.exp(-delta_gr_493/(8.314*493.15))

for i, f in enumerate(ak):
    print('A' + str(i+1) + '=' + str(f) + 'kmol/h')

print('\n')
    
def fun(x_vec): 
    ni = x_vec[:5]
    nt = sum(ni)
    lambdak = x_vec[5:]
    
    s_lambdak_aik = aik.dot(lambdak)
    s_ni_aik = aik.T.dot(ni)
    
    f_1_n = g_493/(8.314 * 493.15)  + np.log(
        ni/nt * 50./1.) + s_lambdak_aik/(8.314 * 493.15)
    f_n_w = - s_ni_aik + ak
    
    return np.concatenate([f_1_n, f_n_w])

ne[ne==0] = np.finfo(float).eps
n0 = ne
x0 = np.concatenate([n0, np.repeat(np.finfo(float).eps, 3)])
sol = optimize.root(fun, x0)
print sol

print('\n')
print('n_i (kmol/h):')
print('\n')
for f in sol.x[:5]:
    print '{:.30g}'.format(f).replace('.',',')
print('\n')
print('lambda_k [adim]:')
print('\n')
for f in sol.x[5:]:
    print '{:.30g}'.format(f).replace('.',',')
print('\n')

A1=60.0kmol/h
A2=70.0kmol/h
A3=200.0kmol/h


    fjac: array([[ -2.92894087e-02,   7.17339066e-03,   7.17338197e-03,
          7.17341674e-03,   7.17341674e-03,   7.06730606e-01,
          7.06730606e-01,  -6.77626358e-21],
       [  5.07180277e-03,  -7.94504615e-03,   5.07532485e-03,
          5.07532485e-03,   5.07532485e-03,   6.81454427e-05,
          6.81454427e-05,   9.99916930e-01],
       [  1.00492194e-01,  -7.03313722e-03,  -1.47887880e-01,
         -7.02977987e-03,  -7.02977987e-03,  -6.92712554e-01,
          6.98592474e-01,   2.56001257e-04],
       [ -7.10067791e-03,   3.30966757e-03,   1.34443406e-02,
         -9.99832940e-01,   1.06560013e-04,   7.95551873e-03,
          1.72751509e-03,   5.06777157e-03],
       [ -5.23373351e-01,  -5.41731532e-01,   2.53832870e-01,
          4.84390044e-03,   5.99430590e-01,  -7.76028721e-02,
          5.27011673e-02,  -6.00361100e-03],
       [  6.93012970e-01,   1.05062668e-01,   5.34421658e-01,
          2.64723665e-03,   4.71892820

c:\users\public\apps\anaconda\lib\site-packages\ipykernel\__main__.py:76: RuntimeWarning: invalid value encountered in log
